### Introduction

##### Experiment setup:
In this notebook I'm presenting a time benchmark which compares the three options to load images for Image Classification task:
1. `tfdata-image-loader` (my repo) 
2. The default Keras `flow_from_directory` option,
3. The new keras.preprocessing `image_dataset_from_directory`

In order to perform the measurements I'm fine tuning a MobilenetV2 head on a new set of classes.  


##### Experiment assumptions

In order to train MobilenetV2 we want to to the following operations:
* resize to (224, 224), 
* normalize values to -1, 1, 
* randomly flip some images, 
* do not cache the content, 
* prefetch newer samples (if possible).

##### Hardware acceleration
For the sake of the experiment I used a GPU. You can choose to use a GPU in `Runtime > Change runtime type > GPU`.

In [ ]:
!nvidia-smi

Fri Mar 19 08:48:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Install TFData Image Loader

In [ ]:
!pip install -e git+git://github.com/sebastian-sz/tfdata-image-loader.git#egg=tfdata-image-loader

Obtaining tfdata-image-loader from git+git://github.com/sebastian-sz/tfdata-image-loader.git#egg=tfdata-image-loader
  Cloning git://github.com/sebastian-sz/tfdata-image-loader.git to ./src/tfdata-image-loader
  Running command git clone -q git://github.com/sebastian-sz/tfdata-image-loader.git /content/src/tfdata-image-loader
  Running setup.py develop for tfdata-image-loader


After installing the external module, please restart your runtime.   
Alternatively you can run:

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

### Download and unpack example dataset:

In [ ]:
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz |tar xzf -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  94.2M      0  0:00:02  0:00:02 --:--:-- 94.2M


Remove the license file so it doesn't mess up our directory tree structure:

In [ ]:
!rm flower_photos/LICENSE.txt

### Setting up our experiment

We are going to define few constants which are going to be helpful during our comparison:

In [ ]:
import os
import pathlib
import time

import numpy as np
import tensorflow as tf

from tfdata_image_loader import TFDataImageLoader

print(f"Is gpu available: {tf.test.is_gpu_available()}")

tf.__version__

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is gpu available: True


'2.4.1'

In [ ]:
EPOCHS = 10
INPUT_SHAPE = (1, 224, 224, 3)

BATCH_SIZE = 32

DATA_PATH = "./flower_photos/"
NUM_CLASSES = len(os.listdir(DATA_PATH))

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

IMG_HEIGHT = IMG_WIDTH = INPUT_SHAPE[1]

### Create Data Loaders 

We are going to create two data loaders:
1. Custom module: `tfdata-image-loader`.
2. The default Keras `flow_from_directory`.
3. New Keras preprocessing `image folder dataset`

#### Create TFData Image Loader and load dataset:

In [ ]:
def preprocess_data(image, label):
    return (image / 127.5) - 1, label


def augment_data(image, label):
    return tf.image.random_flip_left_right(image), label

In [ ]:
train_loader = TFDataImageLoader(
    data_path=DATA_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    pre_process_function=preprocess_data,
    augmentation_function=augment_data,
)

Found 3670 images, belonging to 5 classes

Class names mapping: 
{'daisy': array([1, 0, 0, 0, 0], dtype=int32), 'dandelion': array([0, 1, 0, 0, 0], dtype=int32), 'roses': array([0, 0, 1, 0, 0], dtype=int32), 'sunflowers': array([0, 0, 0, 1, 0], dtype=int32), 'tulips': array([0, 0, 0, 0, 1], dtype=int32)}



In [ ]:
tfdata_image_loader_ds = train_loader.load_dataset()

#### Create Keras Image Loader:

In [ ]:
def preprocess_image(img):
    return (img / 127.5) - 1

train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_image,
    horizontal_flip=True,
)

In [ ]:
keras_data_gen = train_image_generator.flow_from_directory(
    directory=DATA_PATH,
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
)

Found 3670 images belonging to 5 classes.


In [ ]:
train_image_count = len(list(pathlib.Path(DATA_PATH).glob('*/*.jpg')))
train_steps_per_epoch = np.ceil(train_image_count/BATCH_SIZE)

#### Keras Preprocessing `image_dataset_from_directory`:

In [ ]:
image_dataset_from_directory = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_PATH,
  seed=SEED,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE,
  label_mode="categorical"
  )

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)

Found 3670 files belonging to 5 classes.


In [ ]:
image_dataset_from_directory = image_dataset_from_directory.map(lambda x, y: (normalization_layer(x), y)).map(augment_data).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Or try without norm layer (similar results):
# image_dataset_from_directory = image_dataset_from_directory.map(preprocess_data).map(augment_data).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

### Example model declaration
We are going to use a pretrained MobilenetV2 with a custom head.

In [ ]:
def build_model(num_classes):
  base_model = tf.keras.applications.MobileNetV2(
      include_top=False,
      pooling="avg",
      input_shape=INPUT_SHAPE[1:]
  )
  base_model.trainable = False

  return tf.keras.Sequential([
      base_model,
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(num_classes, activation="softmax")             
  ])

In [ ]:
def build_fresh_model(num_classes):
  model = build_model(num_classes=num_classes)
  model.compile(
      optimizer=tf.keras.optimizers.RMSprop(),
      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
      metrics=["accuracy"]      
  )
  return model

### Warmup runs

We are going to make 3 epochs warm up before we make time measurements:

In [ ]:
model = build_fresh_model(NUM_CLASSES)
model.fit(tfdata_image_loader_ds, epochs=3)

9412608/9406464 [==============================] - 0s 0us/step
Epoch 1/3
115/115 [==============================] - 42s 65ms/step - loss: 0.9908 - accuracy: 0.6281
Epoch 2/3
115/115 [==============================] - 7s 62ms/step - loss: 0.3911 - accuracy: 0.8697
Epoch 3/3
115/115 [==============================] - 7s 61ms/step - loss: 0.3057 - accuracy: 0.8963


In [ ]:
model = build_fresh_model(NUM_CLASSES)
model.fit(
    keras_data_gen,
    steps_per_epoch=train_steps_per_epoch,
    epochs=3,
    )

Epoch 1/3
115/115 [==============================] - 16s 122ms/step - loss: 0.9367 - accuracy: 0.6574
Epoch 2/3
115/115 [==============================] - 14s 120ms/step - loss: 0.3886 - accuracy: 0.8610
Epoch 3/3
115/115 [==============================] - 14s 121ms/step - loss: 0.2976 - accuracy: 0.8926


In [ ]:
model = build_fresh_model(NUM_CLASSES)
model.fit(
    image_dataset_from_directory,
    epochs=3,
    )

Epoch 1/3
115/115 [==============================] - 14s 96ms/step - loss: 1.0000 - accuracy: 0.6224
Epoch 2/3
115/115 [==============================] - 11s 95ms/step - loss: 0.3862 - accuracy: 0.8594
Epoch 3/3
115/115 [==============================] - 12s 97ms/step - loss: 0.3266 - accuracy: 0.8857


### Measure performance

After warm up runs we are going to measure the performance of two loaders:

##### Measure time of my TFData Image Loader 

In [ ]:
model = build_fresh_model(NUM_CLASSES)

In [ ]:
start = time.perf_counter()
_ = model.fit(tfdata_image_loader_ds, epochs=EPOCHS)

print(f"Train job took: {time.perf_counter() - start} seconds.")

Epoch 1/10
115/115 [==============================] - 9s 62ms/step - loss: 1.0428 - accuracy: 0.6148
Epoch 2/10
115/115 [==============================] - 7s 63ms/step - loss: 0.3861 - accuracy: 0.8627
Epoch 3/10
115/115 [==============================] - 7s 64ms/step - loss: 0.3070 - accuracy: 0.8980
Epoch 4/10
115/115 [==============================] - 7s 62ms/step - loss: 0.2650 - accuracy: 0.9106
Epoch 5/10
115/115 [==============================] - 7s 62ms/step - loss: 0.2355 - accuracy: 0.9135
Epoch 6/10
115/115 [==============================] - 7s 63ms/step - loss: 0.2206 - accuracy: 0.9282
Epoch 7/10
115/115 [==============================] - 7s 63ms/step - loss: 0.2111 - accuracy: 0.9233
Epoch 8/10
115/115 [==============================] - 7s 63ms/step - loss: 0.1887 - accuracy: 0.9349
Epoch 9/10
115/115 [==============================] - 7s 63ms/step - loss: 0.1848 - accuracy: 0.9382
Epoch 10/10
115/115 [==============================] - 7s 63ms/step - loss: 0.1653 - accura

##### Measure time of the default keras `flow_from_directory`

In [ ]:
model = build_fresh_model(NUM_CLASSES)

In [ ]:
start = time.perf_counter()
_ = model.fit(
    keras_data_gen, 
    steps_per_epoch=train_steps_per_epoch,    
    epochs=EPOCHS
    )

print(f"Train job took: {time.perf_counter() - start} seconds.")

Epoch 1/10
115/115 [==============================] - 16s 122ms/step - loss: 0.9777 - accuracy: 0.6337
Epoch 2/10
115/115 [==============================] - 14s 121ms/step - loss: 0.4121 - accuracy: 0.8527
Epoch 3/10
115/115 [==============================] - 14s 122ms/step - loss: 0.3165 - accuracy: 0.8890
Epoch 4/10
115/115 [==============================] - 14s 120ms/step - loss: 0.2657 - accuracy: 0.9126
Epoch 5/10
115/115 [==============================] - 14s 120ms/step - loss: 0.2404 - accuracy: 0.9214
Epoch 6/10
115/115 [==============================] - 14s 121ms/step - loss: 0.2196 - accuracy: 0.9251
Epoch 7/10
115/115 [==============================] - 14s 120ms/step - loss: 0.2243 - accuracy: 0.9179
Epoch 8/10
115/115 [==============================] - 14s 123ms/step - loss: 0.1923 - accuracy: 0.9366
Epoch 9/10
115/115 [==============================] - 14s 120ms/step - loss: 0.1729 - accuracy: 0.9468
Epoch 10/10
115/115 [==============================] - 14s 120ms/step - l

##### Measure time of the keras.preprocessing `image_dataset_from_directory` loader:

In [ ]:
model = build_fresh_model(NUM_CLASSES)

In [ ]:
start = time.perf_counter()
_ = model.fit(
    image_dataset_from_directory, 
    steps_per_epoch=train_steps_per_epoch,    
    epochs=EPOCHS
    )

print(f"Train job took: {time.perf_counter() - start} seconds.")

Epoch 1/10
115/115 [==============================] - 14s 97ms/step - loss: 1.0087 - accuracy: 0.6178
Epoch 2/10
115/115 [==============================] - 12s 97ms/step - loss: 0.3816 - accuracy: 0.8583
Epoch 3/10
115/115 [==============================] - 11s 95ms/step - loss: 0.3258 - accuracy: 0.8843
Epoch 4/10
115/115 [==============================] - 12s 96ms/step - loss: 0.2794 - accuracy: 0.9042
Epoch 5/10
115/115 [==============================] - 11s 95ms/step - loss: 0.2463 - accuracy: 0.9106
Epoch 6/10
115/115 [==============================] - 11s 95ms/step - loss: 0.2283 - accuracy: 0.9220
Epoch 7/10
115/115 [==============================] - 11s 95ms/step - loss: 0.1994 - accuracy: 0.9314
Epoch 8/10
115/115 [==============================] - 11s 95ms/step - loss: 0.1966 - accuracy: 0.9260
Epoch 9/10
115/115 [==============================] - 11s 95ms/step - loss: 0.1987 - accuracy: 0.9306
Epoch 10/10
115/115 [==============================] - 11s 96ms/step - loss: 0.191

### Conclusions

Using MobilenetV2 as feature extractor for a new classification problem, the time to perform the train job took respectively:
1. 141.66 seconds, when using default Keras loader
2. 117.33 seconds, when using keras.preprocessing `image_dataset_from_directory`
3. 75.37 seconds, when using `tfdata-image-loader` module

The option to use my implementation provided best results in the above use case.